# PyMEOS + MobilityDB

This Notebook contains a demo of a connection between PyMEOS and MobilityDB, separated in 2 sections:
- Read
- Write

In [1]:
host = 'pymeos-demo-db'
port = 5432
db = 'demo'
user = 'docker'
password = 'docker'

In [2]:
from pymeos import *
from pymeos.db import MobilityDB
import pandas as pd
import pandas.io.sql as sqlio
import postgis as pg
from shapely.geometry import Polygon


pymeos_initialize()

## Connection
Use `MobilityDB.connect` from `pymeos.db` instead of `psycopg2.connect` to have access to MobilityDB types.

In [3]:
connection = MobilityDB.connect(host=host, port=port, database=db, user=user, password=password)
cursor = connection.cursor()

## Read from table

Trips table schema:
- vehicle: `INTEGER`
- day: `DATE`
- seq: `INTEGER`
- trip: `TGEOMPOINT`

In [5]:
sql = "SELECT * FROM Trips;"
trips = sqlio.read_sql_query(sql, connection)

/tmp/ipykernel_5669/3703330487.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trips = sqlio.read_sql_query(sql, connection)


In [6]:
trips.head()

,vehicle,day,seq,trip
0,1,2020-06-01,1,[0101000000A5218194E3211E41EB9B43431A1A5941@20...
1,1,2020-06-01,2,[010100000036A6A116E0F51D410818EC5287315941@20...
2,1,2020-06-02,1,[0101000000A5218194E3211E41EB9B43431A1A5941@20...
3,1,2020-06-02,2,[010100000036A6A116E0F51D410818EC5287315941@20...
4,1,2020-06-03,1,[0101000000A5218194E3211E41EB9B43431A1A5941@20...


## Manipulate data

In [15]:
polygon = Polygon([(482500, 6605000), (490000, 6605000), (490000, 6580000), (482500, 6580000)])
postgis_polygon = pg.Geometry.from_ewkb(polygon.wkb_hex)
trips['spatial_cut'] = trips['trip'].apply(lambda trip: trip.at(postgis_polygon))
trips

,vehicle,day,seq,trip,spatial_cut
0,1,2020-06-01,1,[0101000000A5218194E3211E41EB9B43431A1A5941@20...,None
1,1,2020-06-01,2,[010100000036A6A116E0F51D410818EC5287315941@20...,None
2,1,2020-06-02,1,[0101000000A5218194E3211E41EB9B43431A1A5941@20...,None
3,1,2020-06-02,2,[010100000036A6A116E0F51D410818EC5287315941@20...,None
4,1,2020-06-03,1,[0101000000A5218194E3211E41EB9B43431A1A5941@20...,None
5,1,2020-06-03,2,[010100000036A6A116E0F51D410818EC5287315941@20...,None
6,1,2020-06-04,1,[0101000000A5218194E3211E41EB9B43431A1A5941@20...,None
7,1,2020-06-04,2,[010100000036A6A116E0F51D410818EC5287315941@20...,None
8,2,2020-06-01,1,[0101000000F2CC1014306F1D4113BAF7554A1A5941@20...,{[010100000044ACFDFF0F731D4115CACF84241B5941@2...
9,2,2020-06-01,2,[0101000000BBB9695FBC111E419BDF99EFA41D5941@20...,{[01010000001FAC010040E81D418A4F8A46FC1E5941@2...


## Create new table

In [8]:
cursor.execute("""
CREATE TABLE TripsCut(
    vehicle INTEGER,
    day DATE,
    seq INTEGER,
    cut_trip TGEOMPOINT
);""")
connection.commit()

In [12]:
for trip in trips.iterrows():
    t = trip[1]
    cursor.execute(f"INSERT INTO TripsCut(vehicle, day, seq, cut_trip) "
                   f"VALUES ({t['vehicle']}, '{t['day']}', '{t['seq']}', '{t['spatial_cut']}');".replace("'None'", 'NULL'))

In [14]:
connection.commit()
cursor.close()
pymeos_finalize()